In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [59]:
data = pd.read_csv('../data/raw_postpro.csv', encoding = 'cp949')
# 컬럼 삭제
df = data.drop(['Part No.2', '미입고 기간', '선박입고','완료 여부'], axis=1)

In [60]:
X = df[['Subject','Machinery', 'Assembly', 'Part No.1', '청구품목']]
y = df['key2'].values
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
# y = y.values.ravel()  # y를 1차원 배열로 변환
y = le.fit_transform(y)
# 데이터를 학습용과 검증용으로 나누기
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42,shuffle=True)

In [61]:
X_train

,Subject,Machinery,Assembly,Part No.1,청구품목
10862,NET BOAT R/G REPAIR PARTS,7M NET BOAT REDUCTION GEAR,FIG 3. INPUT SHAFT,24321-000450,"O-RING, 1AG45.0"
2088,M/E WATER PUMP 외 PARTS 발주의 건,MAIN ENGINE,LUBE OIL COOLER,8417454,SEAL..O-RING-CLAMP PLATE TO PIPE FLANGE
14882,SEG-E-190513-01,POWER BLOCK,CLEATS SHEAVE ASSY,60.059.01215.134,BUSHING
981,OCA-D-DECK-210410-01,BLOCK,SNATCH BLOCK,109064,MCKISSICK 419 SNATCH BLOCK
6473,선단용 6M NET BOAT O/H PARTS,6M NET BOAT ENGINE,FIG 21. INTER COOLER(6CH-WUTE),127687-18910,"CLIP, HOSE"
...,...,...,...,...,...
11284,SEG-E-201008-01,6M NET BOAT ENGINE,FIG 39. COOLER(FRESH WATER),26226-100752,STUD M10X75
11964,MIR-F-ENG-191015-01,NO.1 GENERATOR ENGINE,307-0008 WIRING GP-ENGINE,304-5668,SENSOR GP-PRESS
5390,체항수리용 REF COMP O/H PARTS 외,NO.1 REF COMPRESSOR,7Z2-1 CYLINDER LINER & CON-ROD,1206570,#2 COMPRESSION RING
860,MAR2-F-ENG-190218-01,PORT BOOM BRAILING WINCH,BRAKE GROUP,25953,DIVIDER PLATE


In [32]:
features= {
   'Subject': df['Subject'].values,
    'Machinery':df['Machinery'].values ,
    'Assembly': df['Assembly'].values,
    'Part No.1' : df['Part No.1'].values,
    '청구품목' : df['청구품목'].values
}

In [62]:
features= {
   'Subject': X_train['Subject'].values,
    'Machinery':X_train['Machinery'].values ,
    'Assembly': X_train['Assembly'].values,
    'Part No.1' : X_train['Part No.1'].values,
    '청구품목' : X_train['청구품목'].values
}

In [63]:
y_train

array([40, 24, 10, ..., 24, 44, 44])

In [47]:
# # df에서 각 컬럼의 단어 수 확인
# colname = ['Subject','Machinery', 'Assembly', 'Part No.1', '청구품목',]
# count = []
# for col in colname:
#     unique_subjects = set(df[col])
#     num_unique_subjects = len(unique_subjects)
#     count.append(num_unique_subjects)
#     # print(num_unique_subjects)
# print(count)

# import pandas as pd

# # 가장 긴 문자열 길이를 저장할 딕셔너리 초기화
# max_string_lengths = {}

# # 각 열 별로 길이가 가장 긴 문자열 찾기
# for col in colname:
#     max_length = df[col].str.len().max()
#     max_string_lengths[col] = max_length
# # 결과 출력
# print(max_string_lengths)

In [48]:
# unique_values = {}
# for col in colname:
#     unique_values[col] = X[col].unique()
#     print(f"{col}: {len(unique_values[col])}")

In [ ]:
string_lookup_layer = tf.keras.layers.StringLookup(vocabulary=['small', 'medium', 'large'], num_oov_indices=0)
embedding = tf.keras.layers.Embedding(3, 4)
print(string_lookup_layer(['small', 'medium', 'large']))
embedding(string_lookup_layer(['small', 'medium', 'large']))

In [31]:
# colname = ['Subject','Machinery', 'Assembly', 'Part No.1', '청구품목',]

inputs = {
      'Subject': tf.keras.Input(shape=(), dtype='string'),
      'Machinery': tf.keras.Input(shape=(), dtype='string'),
      'Assembly': tf.keras.Input(shape=(), dtype='string'),
      'Part No.1': tf.keras.Input(shape=(), dtype='string'),
      '청구품목': tf.keras.Input(shape=(), dtype='string')
    }
# size_output = tf.keras.layers.StringLookup(vocabulary=vocab)(inputs['size'])

subject_output = tf.keras.layers.StringLookup(vocabulary=unique_values['Subject'])(inputs['Subject'])
machinery_output = tf.keras.layers.StringLookup(vocabulary=unique_values['Machinery'])(inputs['Machinery'])
assembly_output = tf.keras.layers.StringLookup(vocabulary=unique_values['Assembly'])(inputs['Assembly'])
part1_output = tf.keras.layers.StringLookup(vocabulary=unique_values['Part No.1'])(inputs['Part No.1'])
item_output = tf.keras.layers.StringLookup(vocabulary=unique_values['청구품목'])(inputs['청구품목'])

outputs = {
      'Subject': subject_output,
      'Machinery':machinery_output,
      'Assembly': assembly_output,
      'Part No.1': part1_output,
      '청구품목': item_output
}
preprocessing_model = tf.keras.Model(inputs, outputs)
# preprocessing_model.summary()

In [56]:
# Apply the preprocessing in tf.data.Dataset.map.
dataset = tf.data.Dataset.from_tensor_slices(features).batch(1)
dataset = dataset.map(lambda x: preprocessing_model(x),
                      num_parallel_calls=tf.data.AUTOTUNE)
# Display a preprocessed input sample.
next(dataset.take(1).as_numpy_iterator())

{'Subject': array([770], dtype=int64),
 'Machinery': array([105], dtype=int64),
 'Assembly': array([1031], dtype=int64),
 'Part No.1': array([4178], dtype=int64),
 '청구품목': array([3390], dtype=int64)}

In [43]:
print(dataset.element_spec)  # 각 요소의 구조를 확인합니다
print(dataset.cardinality().numpy())  # 요소 개수를 확인합니다


{'Subject': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'Machinery': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'Assembly': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'Part No.1': TensorSpec(shape=(None,), dtype=tf.int64, name=None), '청구품목': TensorSpec(shape=(None,), dtype=tf.int64, name=None)}
20517


In [44]:
type(subject_output), subject_output.shape, type(machinery_output), machinery_output.shape,type(assembly_output), assembly_output.shape

(keras.engine.keras_tensor.KerasTensor,
 TensorShape([None]),
 keras.engine.keras_tensor.KerasTensor,
 TensorShape([None]),
 keras.engine.keras_tensor.KerasTensor,
 TensorShape([None]))

In [37]:
inputs = {
      'Subject': tf.keras.Input(shape=(), dtype='string'),
      'Machinery': tf.keras.Input(shape=(), dtype='string'),
      'Assembly': tf.keras.Input(shape=(), dtype='string'),
      'Part No.1': tf.keras.Input(shape=(), dtype='string'),
      '청구품목': tf.keras.Input(shape=(), dtype='string')
    }
# size_output = tf.keras.layers.StringLookup(vocabulary=vocab)(inputs['size'])

embedding1 = tf.keras.layers.Embedding(len(unique_values['Subject']), 30)
embedding2 = tf.keras.layers.Embedding(len(unique_values['Machinery']), 30)
embedding3 = tf.keras.layers.Embedding(len(unique_values['Assembly']), 30)
embedding4 = tf.keras.layers.Embedding(len(unique_values['Part No.1']), 30)
embedding5 = tf.keras.layers.Embedding(len(unique_values['청구품목']), 30)

# embedding = tf.keras.layers.Embedding(len(vocab), embedding_dims)

outputs = tf.keras.layers.Concatenate()([
        embedding1(inputs['Subject']),
        embedding2(inputs['Machinery']),
        embedding3(inputs['Assembly']),
        embedding4(inputs['Part No.1']),
        embedding5(inputs['청구품목'])
])
outputs = tf.keras.layers.Dense(1)(outputs)
training_model = tf.keras.Model(inputs, outputs)
training_model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_41 (InputLayer)          [(None,)]            0           []                               
                                                                                                  
 input_42 (InputLayer)          [(None,)]            0           []                               
                                                                                                  
 input_43 (InputLayer)          [(None,)]            0           []                               
                                                                                                  
 input_44 (InputLayer)          [(None,)]            0           []                               
                                                                                            

In [53]:
# training_model.compile(
#     loss=tf.keras.losses.BinaryCrossentropy(from_logits=True))
training_model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                      optimizer=tf.keras.optimizers.Adam(),
                       metrics=['accuracy'])

# training_model.fit(dataset)
# 모델 학습
training_model.fit(features, y_train, epochs=10, batch_size=32)

Epoch 1/10


D:\ShipSupplies\DA\venv\lib\site-packages\tensorflow\python\util\dispatch.py:1176: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return dispatch_target(*args, **kwargs)


UnimplementedError: Graph execution error:

Detected at node 'model_5/embedding_12/Cast' defined at (most recent call last):
    File "C:\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "D:\ShipSupplies\DA\venv\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "D:\ShipSupplies\DA\venv\lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "D:\ShipSupplies\DA\venv\lib\site-packages\ipykernel\kernelapp.py", line 725, in start
      self.io_loop.start()
    File "D:\ShipSupplies\DA\venv\lib\site-packages\tornado\platform\asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "C:\Python310\lib\asyncio\base_events.py", line 603, in run_forever
      self._run_once()
    File "C:\Python310\lib\asyncio\base_